In [1]:
import flowshape as fs
import igl
import numpy as np
import meshplot as mp
import os
import pandas as pd
from src.utilities.fin_shape_utils import plot_mesh
from src.utilities.fin_class_def import FinData
from src.utilities.functions import path_leaf
import glob2 as glob
import meshplot as mp
import trimesh

from sklearn.decomposition import PCA

import plotly.express as px
import plotly.graph_objects as go

## Load in raw weights and apply rotation correction

### Load SH data

In [2]:
# get list of refined fin mesh objects
root = "/media/nick/hdd02/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/pecfin_dynamics/"
fin_mesh_list = sorted(glob.glob(os.path.join(root, "point_cloud_data", "processed_fin_data", "*smoothed_fin_mesh*")))

# load SH data
sh_dir = os.path.join(root, "point_cloud_data", "SH_analysis", "")
SH_df = pd.read_csv(os.path.join(sh_dir, "fin_sh_df.csv"))

# drop FGF 30hr that is breaking the SH fits 
# SH_df = SH_df.drop(index=37)

y_subdir = os.path.join(sh_dir, "Y_matrices")
y_data_list = sorted(glob.glob(y_subdir + "/*"))

SH_df.head()

,time_index,well_index,estimated_stage_hpf,genotype,nd2_series,chem_i,experiment_date,chem_id,chem_time,scale,...,sh_l029_m049,sh_l029_m050,sh_l029_m051,sh_l029_m052,sh_l029_m053,sh_l029_m054,sh_l029_m055,sh_l029_m056,sh_l029_m057,sh_l029_m058
0,155,2,41.784707,wt,3,NaN,20240619,WT,NaN,85.686056,...,-0.056949,0.042998,-0.066082,-0.003099,0.034600,-0.123294,-0.125870,-0.017210,-0.121879,0.070869
1,0,0,50.000156,wt,1,DMSO_24,20240711_01,DMSO,24.0,61.601334,...,-0.115353,-0.065863,-0.026955,0.032426,0.100551,0.074502,0.045902,0.061831,0.069477,-0.085305
2,0,1,50.005733,wt,2,shh_24,20240711_01,shh,24.0,59.691705,...,-0.007827,0.003944,0.104124,0.017772,0.110996,-0.058319,0.018776,-0.032077,0.050656,-0.185248
3,0,2,50.011715,wt,3,notch_24,20240711_01,notch,24.0,59.526956,...,0.025897,0.014222,0.010021,-0.033158,0.050874,0.008345,0.060315,0.128052,0.013312,0.229254
4,0,3,50.017035,wt,4,Wnt_24,20240711_01,Wnt,24.0,45.595272,...,0.304811,0.193479,0.008246,-0.172476,0.043811,0.002637,-0.116471,-0.035238,0.033676,-0.162896


### Align all datasets relative to a well-behaved reference

In [3]:
def get_sh_colnames(max_degree):

    col_names = []
    l_id_vec = []
    m_id_vec = []
    for l in range(max_degree):
        
        i1 = l**2
        i2 = (l + 1) ** 2

        for m in range(i1, i2):
            col_name = f"sh_l{l:03}_m{m-i1:03}"
            col_names.append(col_name)

            l_id_vec.append(l)
            m_id_vec.append(m-l)
            
    return col_names, l_id_vec, m_id_vec

In [12]:
from tqdm import tqdm 

# extract array of SH coefficients
max_degree = 16
sh_cols, _, _ = get_sh_colnames(max_degree=max_degree)
sh_array = SH_df.loc[:, sh_cols].to_numpy()

# extract reference
well_ref = 46
time_ref = 0
date_ref = "20240711_01"

# get location of the reference
date_ft = SH_df["experiment_date"] == date_ref
well_ft = SH_df["well_index"] == well_ref
time_ft = SH_df["time_index"] == time_ref

ref_row = SH_df.loc[date_ft & well_ft & time_ft, :].reset_index(drop=True)
ref_sh_weights = ref_row.loc[0, sh_cols].to_numpy().astype('complex128')

sh_array_aligned = np.zeros_like(sh_array)
# iterate through and align
for i in tqdm(range(sh_array.shape[0])):
    # get ceoffs
    sh2 = sh_array[i, :].astype('complex128')
    # compute alignment
    rot2 = fs.compute_max_correlation(ref_sh_weights, sh2, l_max=max_degree)
    # apply alignment to weights
    sh_array_aligned[i, :] = fs.rotate_weights(rot2, sh2)

  0%|          | 0/223 [00:00<?, ?it/s]/home/nick/miniconda3/envs/flowshape_env/lib/python3.10/site-packages/flowshape/so3.py:161: ComplexWarning: Casting complex values to real discards the imaginary part
  weights_rot[l**2 : (l + 1) ** 2] = r_mat[l].dot(
 13%|█▎        | 28/223 [00:00<00:00, 274.59it/s]/home/nick/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████| 223/223 [00:00<00:00, 306.72it/s]


In [13]:
SH_df_a = SH_df.copy()
SH_df_a.loc[:, sh_cols] = sh_array_aligned
SH_df_a.to_csv(os.path.join(sh_dir, "fin_sh_df_aligned.csv"), index=False)

In [16]:
print(np.dot(sh_array_aligned[0, :], ref_sh_weights))
print(np.dot(sh_array[0, :], ref_sh_weights))

(18.942025423187513+0j)
(12.809256463194469+0j)
